In [26]:
import keras
keras.__version__

'2.1.6'

In [27]:
!pip install cython

# 6章 テキストとシーケンスのためのディープラーニング

６章ではテキスト、時系列データについて取り上げる。基本的なアルゴリズみはRNNと1D-CNN。    

- 記事のトピックや書籍の著者を識別する文章の分類と時系列データの分類    
- ２つの文章や２つの株式指標がどれくらい深く関連しているか推定する時系列データの比較    
- 英語の文章をフランス語に翻訳するseq2seq    
- ツイートや映画レビューの感情を肯定と否定で感情分類    
- 最近の気象データに基づいて特定の場所の天気を予測する時系列予測    

    
IMDbデータセットで感情分類と気温の予測。をする。

# テキストデータの操作

テキストは最も広く使用されてるシーケンスデータの一つ。文字または単語のシーケンスと解釈。テキストを使って自然言語理解(NLU Natural language understanding)を生成できる。    
テキストの分割に使用できる単位（単語、文字、Nグラム）のことをトークンという。テキストをベクトル化するプロセスは何かしらのトークン化をしている。この数値ベクトルはシーケンステンソルにまとめられてDNNに提供される。    

ベクトルをシーケンスに紐づける方法には何種類かある。    
トークンのone-hotエンコーディング、トークン埋め込み（token embedding）の２つを紹介。    



# 単語と文字の one-hotエンコーディング

one-hotエンコーディングは、トークンをベクトルに変換する一般的手法。



This notebook contains the first code sample found in Chapter 6, Section 1 of [Deep Learning with Python](https://www.manning.com/books/deep-learning-with-python?a_aid=keras&a_bid=76564dff). Note that the original text features far more content, in particular further explanations and figures: in this notebook, you will only find source code and related comments.

----

One-hot encoding is the most common, most basic way to turn a token into a vector. You already saw it in action in our initial IMDB and 
Reuters examples from chapter 3 (done with words, in our case). It consists in associating a unique integer index to every word, then 
turning this integer index i into a binary vector of size N, the size of the vocabulary, that would be all-zeros except for the i-th 
entry, which would be 1.

Of course, one-hot encoding can be done at the character level as well. To unambiguously drive home what one-hot encoding is and how to 
implement it, here are two toy examples of one-hot encoding: one for words, the other for characters.



Word level one-hot encoding (toy example):

In [3]:
import numpy as np
samples = 'The cat sat on the mat.'
#split関数は空白ごとに分割する
print(samples.split())

['The', 'cat', 'sat', 'on', 'the', 'mat.']


In [4]:
import numpy as np

#初期データ。サンプルごとにエントリが１つ含まれている。
#この単純な例ではサンプルは単なる１つの文章だが、文章全体でも良い。
samples = ['The cat sat on the mat.', 'The dog ate my homework.']

# データに含まれている全てのトークンのインデックスを構築
token_index = {}
for sample in samples:
    #１文を単語に分割して、単語を辞書に溜め込む
    #辞書からユニークなワードがないか調べて、未知語は辞書に登録する。
    for word in sample.split():
        if word not in token_index:
            # 文字の辞書とindexを設定する
            token_index[word] = len(token_index) + 1

print(token_index)

{'The': 1, 'cat': 2, 'sat': 3, 'on': 4, 'the': 5, 'mat.': 6, 'dog': 7, 'ate': 8, 'my': 9, 'homework.': 10}


この辞書作成部分はpythonより低レイヤーのc、c++で書いたり、並列化しやすいjava、scala、goなどで書いた方がいい場合もあると思う。

せっかくの休みなんでcython(サイソン)使ってみる。
    
https://qiita.com/kenmatsu4/items/7c08a85e41741e95b9ba

In [0]:
!pip install cython

In [28]:
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


%%cythonをはじめに書くとjupyterでcythonがかけるよう。-nで名前をつけとくと後から便利らしい。

In [0]:
%%cython -n test_cython_code
def fib(int n):
    cdef int i
    cdef double a=0.0, b=1.0

    for i in range(n):
        a, b = a+b, a
    return a

def primes(int kmax):
    cdef int n, k, i
    cdef int p[1000]
    result = []

    if kmax > 1000:
        kmax = 1000

    k = 0
    n = 2
    while k < kmax:
        i = 0
        while i < k and n % p[i] != 0:
            i += 1

        if i == k:
            p[k] = n
            k += 1
            result.append(n)
        n += 1
    return result

In [9]:
print(fib(90))
print(primes(20))

2.880067194370816e+18
[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71]


とりま動いた。

In [0]:
import numpy as np

samples = ['The cat sat on the mat.', 'The dog ate my homework.']
def make_token_index_py():
    token_index = {}
    for sample in samples:
        for word in sample.split():
            if word not in token_index:
                token_index[word] = len(token_index) + 1

In [40]:
timeit -n2 -r3 make_token_index_py()

2 loops, best of 3: 3.17 µs per loop


In [0]:
%%cython -n make_token_index_1

import numpy as np
cimport numpy as np #cimport使用

samples = ['The cat sat on the mat.', 'The dog ate my homework.']
def make_token_index_cy():
    token_index = {}
    for sample in samples:
        for word in sample.split():
            if word not in token_index:
                token_index[word] = len(token_index) + 1

In [41]:
timeit -n2 -r3 make_token_index_cy()

2 loops, best of 3: 1.68 µs per loop


何も書き換えてないのに早いじゃないか。

In [48]:
!ls

datalab


In [0]:
%%cython -n make_token_index_2

import numpy as np
cimport numpy as np

cdef list samples = ['The cat sat on the mat.', 'The dog ate my homework.']
# samples = ['The cat sat on the mat.', 'The dog ate my homework.']
def make_token_index_cy_def_type():
    cdef dict token_index = {}
#     token_index = {}
    for sample in samples:
        for word in sample.split():
            if word not in token_index:
                token_index[word] = len(token_index) + 1

In [52]:
timeit -n2 -r3 make_token_index_cy_def_type()

NameError: ignored

型指定しようとしたけど、なんかエラってる。一旦置いておこう。

トークン一覧をtoken_indexに格納して、ワンホットエンコーディングする

In [54]:
import numpy as np
samples = ['The cat sat on the mat.', 'The dog ate my homework.']
token_index = {}
for sample in samples:
    for word in sample.split():
        if word not in token_index:
            token_index[word] = len(token_index) + 1
print(token_index)

#次にサンプルをベクトル化する
#サンプルごとに最初のmax_length個の単語だけを考慮
max_length = 10

#結果の格納場所
results = np.zeros((len(samples), max_length, max(token_index.values()) + 1))
print(results.shape)
print(results)

for i, sample in enumerate(samples):
    for j, word in list(enumerate(sample.split()))[:max_length]:
        index = token_index.get(word)
        results[i, j, index] = 1.

print(results)

{'The': 1, 'cat': 2, 'sat': 3, 'on': 4, 'the': 5, 'mat.': 6, 'dog': 7, 'ate': 8, 'my': 9, 'homework.': 10}
(2, 10, 11)
[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]]
[[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 1. 0. 0. 0.

In [55]:
import string
print(repr(string.printable))

'0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ \t\n\r\x0b\x0c'


### 文字レベルでの単純なone-hotエンコーディング

In [59]:
import string

samples = ['The cat sat on the mat.', 'The dog ate my homework.']
characters = string.printable  # All printable ASCII characters.
#スマートな書き方の印象
#zipでset型とrangeからでたlistが回って、dictにまとめてる。
token_index = dict(zip(characters, range(1, len(characters) + 1)))
print("token_index", list(token_index)[:10])

max_length = 50
#ここもzerosのshapeをsamplesのlenとmax_lenghtとtoken_indexのmaxで作ってる。
#先にshapeを作ってるのでcythonにしても早そう。
results = np.zeros((len(samples), max_length, max(token_index.values()) + 1))
for i, sample in enumerate(samples):
    for j, character in enumerate(sample[:max_length]):
        index = token_index.get(character) #getでdictからindexを抜き出す
        results[i, j, index] = 1. #ワンホットになる
print(results[0])

token_index ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


Note that Keras has built-in utilities for doing one-hot encoding text at the word level or character level, starting from raw text data. 
This is what you should actually be using, as it will take care of a number of important features, such as stripping special characters 
from strings, or only taking into the top N most common words in your dataset (a common restriction to avoid dealing with very large input 
vector spaces).

### Kerasを使った単語レベルでのone-hotエンコーディング

kerasには便利なtoknizer関数がある。

In [60]:
from keras.preprocessing.text import Tokenizer

samples = ['The cat sat on the mat.', 'The dog ate my homework.']

# 出現頻度が最も高い1000個の単語だけ処理する
#トークナイザー作成
tokenizer = Tokenizer(num_words=1000)
#単語インデックス構築
tokenizer.fit_on_texts(samples)

#文字列の整数のインデックスのリストに変換
sequences = tokenizer.texts_to_sequences(samples)

print(sequences)

[[1, 2, 3, 4, 1, 5], [1, 6, 7, 8, 9]]


In [61]:
#2値のone-hotエンコーディング表現を取得。one-hot以外のモードもある。
one_hot_results = tokenizer.texts_to_matrix(samples, mode='binary')
print("one_hot_results ", one_hot_results)

# tokenizerが計算した単語のインデックスを復元
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

one_hot_results  [[0. 1. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]]
Found 9 unique tokens.


‘binary‘: デフォルト。文章に存在するかどうかのみ。    
‘count‘: 文字のカウント。    
‘tfidf‘: 全体頻度割る文字のカウント。    
‘freq‘: 全体単語頻度。    

In [62]:
#いろんなtexts_to_matrix
for mode in ['binary', 'count', 'tfidf', 'freq']:
    matrix = tokenizer.texts_to_matrix(samples, mode) 
    print("matrix ", "mode: ", mode, matrix)

matrix  mode:  binary [[0. 1. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]]
matrix  mode:  count [[0. 2. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]]
matrix  mode:  tfidf [[0.         0.86490296 0.69314718 ... 0.         0.         0.        ]
 [0.         0.51082562 0.         ... 0.         0.         0.        ]]
matrix  mode:  freq [[0.         0.33333333 0.16666667 ... 0.         0.         0.        ]
 [0.         0.2        0.         ... 0.         0.         0.        ]]


text_to_word_sequenceとsetの組み合わせ便利そうだけど、語彙増えると分割しないとメモリで死にそう

In [74]:
#https://machinelearningmastery.com/prepare-text-data-deep-learning-keras/
from keras.preprocessing.text import text_to_word_sequence
text = 'The quick brown fox jumped over the lazy dog.'
words = set(text_to_word_sequence(text))
vocab_size = len(words)
print(words)
print(vocab_size)

{'jumped', 'the', 'fox', 'dog', 'lazy', 'over', 'brown', 'quick'}
8


one_hot関数ってのもある

In [75]:
from keras.preprocessing.text import one_hot
from keras.preprocessing.text import text_to_word_sequence
text = 'The quick brown fox jumped over the lazy dog.'
words = set(text_to_word_sequence(text))
vocab_size = len(words)
print(vocab_size)
result = one_hot(text, round(vocab_size*1.3))
print(result)

8
[2, 7, 5, 2, 2, 1, 2, 8, 3]


hashing_trick。md5指定版。

In [77]:
from keras.preprocessing.text import hashing_trick
from keras.preprocessing.text import text_to_word_sequence
text = 'The quick brown fox jumped over the lazy dog.'
words = set(text_to_word_sequence(text))
vocab_size = len(words)
print(vocab_size)
#2個目の引数がハッシュ空間の次元。結果を見ると早速index被ってない？
result = hashing_trick(text, round(vocab_size*1.3), hash_function='md5')
print(result)
result = hashing_trick(text, round(vocab_size*2), hash_function='md5')
print(result)
result = hashing_trick(text, round(vocab_size*3), hash_function='md5')
print(result)

8
[6, 4, 1, 2, 7, 5, 6, 2, 6]
[15, 10, 13, 14, 10, 8, 15, 8, 6]
[18, 20, 16, 12, 22, 3, 18, 6, 17]


### Toknizerの保存と読み込み

トークンが多くなったら毎回トークナイズしたくない場合もある。その場合は何かしらシリアライズしたい。    

tokenizerをpickleで塩漬けにする方法

In [63]:
from keras.preprocessing.text import Tokenizer

samples = ['The cat sat on the mat.', 'The dog ate my homework.']
tokenizer = Tokenizer(num_words=1000)
tokenizer.fit_on_texts(samples)
sequences = tokenizer.texts_to_sequences(samples)
results_vec = tokenizer.texts_to_matrix(samples, mode='binary')
word_index = tokenizer.word_index
print("sequences", sequences) #単語ID一覧
print("results_vec", results_vec) #ワンホット

sequences [[1, 2, 3, 4, 1, 5], [1, 6, 7, 8, 9]]
results_vec [[0. 1. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]]


In [0]:
import pickle
# saving
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [67]:
!ls

datalab  tokenizer.pickle


In [0]:
# loading
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [66]:
sequences = tokenizer.texts_to_sequences(samples)
results_vec = tokenizer.texts_to_matrix(samples, mode='binary')
word_index = tokenizer.word_index
print("sequences", sequences) #単語ID一覧
print("results_vec", results_vec) #ワンホット

sequences [[1, 2, 3, 4, 1, 5], [1, 6, 7, 8, 9]]
results_vec [[0. 1. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]]


毎回データが同じなんですって場合はベクトルの方を保存してもいいと思う。

In [0]:
from keras.preprocessing.text import Tokenizer

samples = ['The cat sat on the mat.', 'The dog ate my homework.']
tokenizer = Tokenizer(num_words=1000)
tokenizer.fit_on_texts(samples)
sequences = tokenizer.texts_to_sequences(samples)
results_vec = tokenizer.texts_to_matrix(samples, mode='binary')
word_index = tokenizer.word_index
np.save(open("sequences.npy", 'wb'), sequences)
np.save(open("results_vec.npy", 'wb'), results_vec)

In [71]:
!ls

datalab      results_vec.npy  sequences.npy
results_vec  sequences	      tokenizer.pickle


In [72]:
sequences2 = np.load("sequences.npy")
results_vec2 = np.load("results_vec.npy")
print("sequences", sequences2) #単語ID一覧
print("results_vec", results_vec2) #ワンホット

sequences [list([1, 2, 3, 4, 1, 5]) list([1, 6, 7, 8, 9])]
results_vec [[0. 1. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]]


エンコーディング方法にone-hotハッシュトリックというものもある。役立つのは語彙に含まれている一意のトークンの数がそのまま処理するには多すぎる場合。    
単語をインデックス参照する代わりに、単語を固定サイズのベクトルにハッシュ化する。    
軽量ハッシュを使用。単語インデックスを明示的に保持しないでメモリを節約する。データのオンラインコーディングを可能にする。    
利用可能なデータが全て揃う前に、トークンベクトルを生成できる。    

欠点はハッシュ衝突の恐れがある。    

ハッシュ衝突の可能性が低くなるのは、ハッシュ化の対象となる一意なトークンの総数よりもハッシュ空間の次元数の方が大きい場合。    
つまりハッシュ空間でかくしとけってか。    



### ハッシュトリックを用いた単語レベルの単純なone-hotエンコーディング

kerasの関数使えばいいけど、直に書くとこの雰囲気。

In [78]:
samples = ['The cat sat on the mat.', 'The dog ate my homework.']

#単語をサイズが1000ベクトルとして格納なのでハッシュ空間が1000
#ボキャブラリ数が1000に近くなるとハッシュ衝突が発生する。
dimensionality = 1000
max_length = 10

results = np.zeros((len(samples), max_length, dimensionality))
for i, sample in enumerate(samples):
    for j, word in list(enumerate(sample.split()))[:max_length]:
        #単語をハッシュ化して、0〜1000のランダム整数に変換
        index = abs(hash(word)) % dimensionality
        results[i, j, index] = 1.
        
print(results)

[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]


hashing_trickは多分こんな感じ？

In [80]:
from keras.utils import to_categorical
from keras.preprocessing.text import hashing_trick
from keras.preprocessing.text import text_to_word_sequence
samples = ['The cat sat on the mat.', 'The dog ate my homework.']
dimensionality = 1000
max_length = 10

for i, sample in enumerate(samples):
    #単語をハッシュ化して、0〜1000のランダム整数に変換
    result = hashing_trick(sample, dimensionality, hash_function='md5')
    print(result)
    print(to_categorical(result))

[132, 759, 622, 379, 132, 962]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]]
[132, 168, 822, 28, 128]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
